In [104]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
!pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier
import matplotlib.pyplot as pyplot

Error initializing plugin EntryPoint('Windows (alt)', 'keyrings.alt.Windows', None, Distribution('keyrings.alt', '3.0')).
Traceback (most recent call last):
  File "/home/nyle/.local/lib/python3.6/site-packages/keyring/backend.py", line 190, in _load_plugins
    init_func = ep.load()
  File "/home/nyle/.local/lib/python3.6/site-packages/entrypoints.py", line 82, in load
    mod = import_module(self.module_name)
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 955, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 665, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr

In [105]:
# Reading shopping data
X_train = pd.read_csv('online_jewellery_shop_train.csv')
X_final = pd.read_csv('online_jewellery_shop_test_final.csv')

idd_final = X_final.pop("ID")
idd = X_train.pop("ID")
idd.head()
df = X_train.copy()
df.head()

df_final = X_final.copy()


##  variables for categorical data and cleaning data

In [106]:
dummiesdf = pd.get_dummies(df['VisitorType'])
df.drop('VisitorType', inplace = True, axis = 1)
df['New_Visitor'] = dummiesdf['New_Visitor']
df['Other'] = dummiesdf['Other']
df['Returning_Visitor'] = dummiesdf['Returning_Visitor']

dummiesdf_final = pd.get_dummies(df_final['VisitorType'])
df_final.drop('VisitorType', inplace = True, axis = 1)
df_final['New_Visitor'] = dummiesdf_final['New_Visitor']
df_final['Other'] = dummiesdf_final['Other']
df_final['Returning_Visitor'] = dummiesdf_final['Returning_Visitor']

In [107]:
dfmonth = pd.get_dummies(df['Month']) 
df.drop('Month', inplace = True, axis = 1)
dfwithdummies = pd.concat([df, dfmonth], axis = 1, sort = False)

dfmonth_final = pd.get_dummies(df_final['Month']) 
df_final.drop('Month', inplace = True, axis = 1)
dfwithdummies_final = pd.concat([df_final, dfmonth_final], axis = 1, sort = False)


In [108]:
dfwithdummies['Class'] = df['Revenue'].astype(int)
dfwithdummies.drop('Revenue', axis = 1, inplace = True)
dfwithdummies['Weekend'] = df['Weekend'].astype(int)
dfwithdummies.drop('Returning_Visitor', axis = 1, inplace = True)
dfcleaned = dfwithdummies.copy()


dfwithdummies_final['Weekend'] = df_final['Weekend'].astype(int)
dfwithdummies_final.drop('Returning_Visitor', axis = 1, inplace = True)
dfcleaned_final = dfwithdummies_final.copy()

In [109]:
X = dfcleaned.drop('Class', axis = 1)
Y = dfcleaned['Class'].copy()

X_F = dfcleaned_final


cols_for_dummying = [ 'OperatingSystems', 'Browser', 'Region', 'TrafficType']

for col in cols_for_dummying: # stringify
        X[col] = X[col].apply (str)

for col in cols_for_dummying: # stringify
        X_F[col] = X_F[col].apply (str)

X = pd.get_dummies (X, prefix_sep = '_', drop_first = True)
X_F = pd.get_dummies (X_F, prefix_sep = '_', drop_first = True)

X_F.drop(['Browser_9', 'TrafficType_12'], axis = 1, inplace = True)
X_F["TrafficType_17"] = 0

print(X.shape)
print(X_F.shape)

(8630, 67)
(3700, 67)


## Collinearity Between Features 

In [110]:
# Need to clean the data to make sure in range
# Checking for NA values

X_org = X
Y_org = Y

print(X.shape) 
result = pd.concat([X, Y], axis=1, sort=False)
result.shape
result = result[(result >= 0).all(1)]
print(result.shape)
X = result.drop('Class', axis = 1)
Y = result['Class'].copy()
print(X.shape)
print(Y.shape)

from collections import defaultdict
avgs = defaultdict(int)

for k, v in X.iteritems(): 
    avgs[k] = v.mean()


# Aggreagte all of data mean
resultXF = X_F.copy()
resultXF = resultXF[(resultXF >= 0).all(1)]
for k, v in resultXF.iteritems(): 
    avgs[k] =  (avgs[k] * 0.700236) + (v.mean() * 0.29976)

print("resultXF", resultXF.shape)


#print(X_org.ix[66])  
########
for k, v in X_org.iteritems(): 
    v[v < 0] = 0
#print(X_org.ix[66]) 

#print(X_org.ix[502]) 
for column in X_org.columns: 
    X_org[column] = X_org[column].fillna(avgs[column])
#print(X_org.ix[502]) 
    
X = X_org
Y = Y_org
print(X.shape)
print(Y.shape)

#print(X_org.ix[66])  
########
for k, v in X_F.iteritems(): 
    v[v < 0] = 0
#print(X_org.ix[66]) 

#print(X_org.ix[502]) 
for column in X_F.columns: 
    X_F[column] = X_F[column].fillna(avgs[column])
#print(X_org.ix[502]) 

print("X_F",X_F.shape)
# Include the final in the mean calc -> would improve your score more
# After



(8630, 67)
(8601, 68)
(8601, 67)
(8601,)
resultXF (3682, 67)


/home/nyle/.local/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(8630, 67)
(8630,)


/home/nyle/.local/lib/python3.6/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


X_F (3700, 67)


In [111]:
# dfcleaned[dfcleaned['New_Visitor'] == 1]['Administrative'].value_counts()
def AvgMinutes(Count, Duration):
    if Duration == 0:
        output = 0
    elif Duration != 0:
        output = float(Duration)/float(Count)
    return output

def AvgExitBounceRates(Bounce, Exit):
    return ((float(Bounce) + float(Exit)) / 2.0)



Columns = [['Administrative', 'Administrative_Duration'], ['Informational', 'Informational_Duration'], ['ProductRelated', 'ProductRelated_Duration'], ['BounceRates', 'ExitRates']] 
    
X['AvgAdministrative'] = X.apply(lambda x: AvgMinutes(Count = x['Administrative'], Duration = x['Administrative_Duration']), axis = 1)
X['AvgInformational'] = X.apply(lambda x: AvgMinutes(Count = x['Informational'], Duration = x['Informational_Duration']), axis = 1)
X['AvgProductRelated'] = X.apply(lambda x: AvgMinutes(Count = x['ProductRelated'], Duration = x['ProductRelated_Duration']), axis = 1)
X['AvgBounceExit'] = X.apply(lambda x: AvgExitBounceRates(Bounce = x['BounceRates'], Exit = x['ExitRates']), axis = 1)
X.drop(['Administrative', 'Administrative_Duration','Informational', 'Informational_Duration','ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates'], axis = 1, inplace = True)


X_F['AvgAdministrative'] = X_F.apply(lambda x: AvgMinutes(Count = x['Administrative'], Duration = x['Administrative_Duration']), axis = 1)
X_F['AvgInformational'] = X_F.apply(lambda x: AvgMinutes(Count = x['Informational'], Duration = x['Informational_Duration']), axis = 1)
X_F['AvgProductRelated'] = X_F.apply(lambda x: AvgMinutes(Count = x['ProductRelated'], Duration = x['ProductRelated_Duration']), axis = 1)
X_F['AvgBounceExit'] = X_F.apply(lambda x: AvgExitBounceRates(Bounce = x['BounceRates'], Exit = x['ExitRates']), axis = 1)
X_F.drop(['Administrative', 'Administrative_Duration','Informational', 'Informational_Duration','ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates'], axis = 1, inplace = True)



## Normalize

In [112]:
# Scaling to normalize data
X_copy = X.copy()
rc = RobustScaler()
X_rc=rc.fit_transform(X_copy)
X_rc=pd.DataFrame(X_rc,columns=X.columns)

    
# Scaling to normalize data
X_Fcopy = X_F.copy()
X_Frc= rc.fit_transform(X_Fcopy)
X_Frc=pd.DataFrame(X_Frc,columns=X_F.columns)


## Finding Important Features then Removing from Dataframe

In [113]:
from sklearn import svm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
list_one =[]
feature_ranking = SelectKBest(chi2, k=5)
fit = feature_ranking.fit(X, Y)

fmt = '%-8s%-20s%s'

for i, (score, feature) in enumerate(zip(feature_ranking.scores_, X.columns)):
    list_one.append((score, feature))
    
dfObj = pd.DataFrame(list_one) 
dfObj.sort_values(by=[0], ascending = False)

,0,1
0,118526.635936,PageValues
60,7496.799478,AvgInformational
59,4334.128248,AvgAdministrative
61,599.350140,AvgProductRelated
12,156.506153,Nov
50,79.791768,TrafficType_2
3,75.949859,New_Visitor
52,57.287583,TrafficType_3
16,46.567623,OperatingSystems_3
43,40.832226,TrafficType_13


In [114]:
rem = list(set (X_rc.columns) - set(dfObj.nlargest (28, 0)[1]))
rem
X_rc.drop(rem,axis=1,inplace=True)
X_Frc.drop(rem,axis=1,inplace=True)

print(X_rc.shape)
print(X_Frc.shape)

(8630, 28)
(3700, 28)


## XGBoost Classifier with Feature Selection Dataframe

In [ ]:


test_rou = []
train_rou = []

for i in range(100):
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_rc,Y,test_size=0.9)

    model1 = XGBClassifier(learning_rate = 0.1, n_estimators=200, min_child_weight=4, max_depth=4, gamma=0.2, colsample_bytree=1)
    eval_set = [(X_test1, y_test1)]
    model1.fit(X_train1, y_train1, verbose=False)


    y_roc = model1.predict_proba(X_test1)
    y_roc_train = model1.predict_proba(X_train1)

    test_rou.append(roc_auc_score(y_test1, y_roc[:,  1]))
    train_rou.append(roc_auc_score(y_train1, y_roc_train [:,  1]))
    
    yF_roc = model1.predict_proba(X_Frc)
    Revenue = pd.DataFrame(yF_roc[:, 1])
    Revenue.columns = ["Revenue"]
    ID = pd.DataFrame(idd_final)
    out = pd.concat([ID["ID"], Revenue["Revenue"]], axis=1, sort=False)
    out.to_csv("submission.csv",index=False)


print(sum(train_rou)/len(train_rou))
print(sum(test_rou)/len(test_rou))










In [ ]:

def runmodel(lr,md,mcw,g,cb):
    train_rou = []
    test_rou = []

    for i in range(9):
        X_train1, X_test1, y_train1, y_test1 = train_test_split(X_rc,Y,test_size=.1)

        model1 = XGBClassifier(learning_rate = lr, n_estimators=55, min_child_weight=mcw, max_depth=md, gamma=g, colsample_bytree=cb)
        eval_set = [(X_test1, y_test1)]
        model1.fit(X_train1, y_train1, verbose=False)


        y_roc = model1.predict_proba(X_test1)
        y_roc_train = model1.predict_proba(X_train1)

        # Out
        yF_roc = model1.predict_proba(X_Frc)
        Revenue = pd.DataFrame(yF_roc[:, 1])
        Revenue.columns = ["Revenue"]
        ID = pd.DataFrame(idd_final)
        out = pd.concat([ID["ID"], Revenue["Revenue"]], axis=1, sort=False)
        out.to_csv("submission.csv",index=False)


        train_rou.append(roc_auc_score(y_train1, y_roc_train[:,  1]))
        test_rou.append(roc_auc_score(y_test1, y_roc[:,  1]))

    return float(sum(test_rou)/9)



lr = [0.05, 0.10, 0.15, 0.20] 
md = [ 3, 4, 5, 6]
mcw = [ 1, 3, 5, 7 ]
g = [ 0.0, 0.1, 0.2 , 0.3, 0.4]
cb = [ 0.3, 0.4, 0.5 , 0.7 ] 




ps = []
maxtest = []


for a in lr:
    for b in md:
        for c in mcw:
            for d in g:
                for e in cb:
                    di = runmodel(a, b, c,d, e)
                    maxtest.append(di)
                    
                    parameters = defaultdict()
                    parameters["a"] = a
                    parameters["b"] = b
                    parameters["c"] = c
                    parameters["d"] = d
                    parameters["e"] = e
                    ps.append(parameters)
                        

i = maxtest.index(max(maxtest))                        
print(ps[i])
print(maxtest[i])
                        
# NEED TO APPEND ID

